In [12]:
import os
os.environ["CORENLP_HOME"] = r'C:\stanford-corenlp-full-2018-10-05'

import corenlp 
client = corenlp.CoreNLPClient()

def get_nouns(sentence, chunk = False):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:NN}] | [{pos:NNS}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err')
        res = []
    return res

def get_adverbs(sentence, chunk = False):
    text = sentence.lower()
    res = []
    try:
        pattern = '[{pos:RB}]'
        matches = client.tokensregex(sentence, pattern)
        response = matches['sentences'][0]
        len_chunk = response['length']
        if len_chunk > 0:
            for index in range(0, len_chunk):
                res.append(response[str(index)]['text'])
    except:
        print('err')
        res = []
    return res

In [13]:
import pandas as pd
from tqdm import tqdm_notebook

def run(tipe):
    df = pd.read_csv("lda2vec works/lda2vec"+ tipe +".csv")
    sf = pd.DataFrame(columns = ['id','review','category','term'])
    count = 0
    for index in tqdm_notebook(range(len(df))):
        adverbs = get_adverbs(df['review'][index])
        new_term = []
        if type(df['term'][index]) != float:
            terms = df['term'][index].split('|')
            for term in terms:
                if term not in adverbs:
                    new_term.append(term)
                
        if len(new_term) == 0:
            new_term = '|'.join(get_nouns(df['review'][index]))
            count += 1
        else:
            new_term = df['term'][index]
        sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'category': df['category'][index],
                    'term': new_term
                   }, ignore_index=True)
    print(count)

    sf.to_csv("lda2vec works/lda2vec"+ tipe +"-noun.csv")
    sf.to_excel("lda2vec works/lda2vec"+ tipe +"-noun.xlsx")

In [14]:
import pandas as pd
from tqdm import tqdm_notebook

def chunk(filename):
    df = pd.read_csv(filename)
    sf = pd.DataFrame(columns = ['id','review','category','term'])
    
    for index in tqdm_notebook(range(len(df))):
        new_term = '|'.join(get_nouns(df['review'][index]))
        sf = sf.append({'id': df['id'][index], 
                    'review': df['review'][index],
                    'category': df['category'][index],
                    'term': new_term
                   }, ignore_index=True)

    sf.to_csv("Results/Aspect Terms Extraction/chunk.csv")
    sf.to_excel("Results/Aspect Terms Extraction/chunk.xlsx")

In [15]:
chunk('Results/Aspect Terms Extraction/20.csv')

HBox(children=(IntProgress(value=0, max=699), HTML(value='')))